<a href="https://colab.research.google.com/github/Aapng-cmd/fish/blob/master/voice_synth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
import librosa.display
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [107]:
from google.colab import files
files.upload()

!mv sample*.wav audio
NIGGA = 4

Saving sample0.wav to sample0.wav
Saving sample1.wav to sample1.wav
Saving sample2.wav to sample2.wav
Saving sample3.wav to sample3.wav


In [108]:
import speech_recognition as sr
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import nltk
from nltk.tokenize import sent_tokenize

# nltk.download('punkt')  # download the Punkt tokenizer models

def transcribe_audio(path):
    with sr.AudioFile(path) as source:
        audio_listened = r.record(source)
        try:
            text = r.recognize_google(audio_listened, language="ru-RU")
        except sr.UnknownValueError as e:
            print("Error:", str(e))
            text = ""
        return text

def get_large_audio_transcription_on_silence(path):
    sound = AudioSegment.from_file(path)
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=sound.dBFS-14, keep_silence=500)
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        text = transcribe_audio(chunk_filename)
        whole_text += text + " "
    sentences = sent_tokenize(whole_text)
    return ". ".join(sentences)

r = sr.Recognizer()
for i in range(NIGGA):
    path = f"audio/sample{i}.wav"
    text = get_large_audio_transcription_on_silence(path)
    with open(f"data/sample{i}.txt", "w") as f:
        f.write(text)

    print(text)

с доступностью игр в России всё очень-очень сложно тем не менее компания пытается сохранять лояльную аудиторию
вечером ты помнишь вьюга злилась на мутном небе мгла носилась Луна как бледное пятно сквозь тучи мрачные желтела
под голубыми небесами великолепными коврами блестя на Солнце снег лежит
тогда менеджер сообщество под ником дал понять что речь не идёт о какой-то целенаправленной блокировки игроков из России


In [128]:
class TextDataset(Dataset):
    def __init__(self, texts, audio_waveforms):
        self.texts = texts
        self.audio_waveforms = audio_waveforms

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        audio_waveform = self.audio_waveforms[idx]
        # Preprocess the text into a numerical representation
        text_tensor = self.text_to_tensor(text)
        return text_tensor, audio_waveform

    def text_to_tensor(self, text):
        # Tokenize the text into subwords
        tokens = ['<sos>'] + text.split() + ['<eos>']
        # Pad the sequence to a fixed length
        max_seq_len = 100
        tokens = tokens[:max_seq_len]
        tokens += ['<pad>'] * (max_seq_len - len(tokens))
        # Embed each token into a vector
        embedding_dim = 128
        token_embedding = nn.Embedding(num_embeddings=10000, embedding_dim=embedding_dim)
        token_ids = torch.tensor([ord(c) for c in ' '.join(tokens)])
        text_tensor = token_embedding(token_ids)
        return text_tensor  # Remove the batch dimension

In [129]:
# Create a DataLoader to load the data in batches
batch_size = 32
train_data = TextDataset([f'audio/sample{i}.wav' for i in range(NIGGA)], [f'data/sample{i}.txt' for i in range(NIGGA)])
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)


In [130]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Permute to (batch_size, embedding_dim, seq_len)
        x = self.conv1d(x)
        x = self.relu(x)
        return x

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=3)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv1d(x)
        x = self.relu(x)
        return x

class Tacotron(nn.Module):
    def __init__(self):
        super(Tacotron, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, text_tensor):
        encoder_output = self.encoder(text_tensor)
        decoder_output = self.decoder(encoder_output)
        return decoder_output

In [131]:
# Initialize the model and optimizer
model = Tacotron()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.MSELoss()

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (text_tensors, audio_waveforms) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(text_tensors)
        loss = criterion(output, audio_waveforms)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(text_tensors), len(train_data),
                100. * batch_idx / len(train_loader), loss.item()))

print('Training complete!')

AttributeError: 'tuple' object has no attribute 'size'

In [114]:
# Save the model after training
torch.save(model.state_dict(), 'models/fastspeech.pt')

In [116]:
!ls models
files.download("models/fastspeech.pt")

fastspeech.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>